In [1]:
import gzip
from correctionlib.schemav2 import CorrectionSet,Correction,Category,CategoryItem, Formula, MultiBinning
from itertools import chain

In [2]:
def getdict(file):
    import yaml
    # opening a file
    with open(file, 'r') as stream:
        try:
            d=yaml.safe_load(stream)
        except yaml.YAMLError as e:
            print(e)
    #k={"2018":{"HLT_Photon120": {315257: {1: 49}, 315259: {1: 49}, 315264: {32: 74}, 315265: {4: 74}}}}
    return d

In [3]:
def dictf(prescale_dict,year,HLT,run,lumi):
    try:
        dd=prescale_dict[year][HLT][run][lumi]
    except:
        dd=-1
    return dd
 

In [10]:
import copy
def getmultibinning(triggerlumilist):
    triggerlumilist1=copy.deepcopy(triggerlumilist)#.deepcopy()
    if triggerlumilist[0]!=1:
        triggerlumilist1.insert(0, 1)
    tot=len(triggerlumilist1)
    triggerlumilist1.insert(tot, 99999)
    return triggerlumilist1

def prescaletrick(prescalelist,lumilist_):
    if lumilist_[0]!=1:
        prescalelist.insert(0, 1)
    return prescalelist

def getvalue(prescale_dict,year,HLT,run):
    print(year,HLT,run)
    lumilist_= list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT][run]])))
    print(lumilist_)
    print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    print(lumilist_)
    print(getmultibinning(lumilist_))
    print(lumilist_)
    print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    print(prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_))
    value=MultiBinning.parse_obj({
        "inputs":["lumi"],
        "nodetype": "multibinning",
        "edges": [
            getmultibinning(lumilist_)
        ],
        "content": prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_)  ,
        "flow": 'error',
    })
    print(prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_))
    return value

def getdictpre(years):
    prescale_dict={'2018':getdict("2018_UL/photon_prescales_2018.yaml.txt")}
    
    HLTs = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year] for year in years])))
    #for year in years:
    #    runs = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT] for HLT in HLTs])))
    #for year in years:
    #    for HLT in HLTs:
    #        lumis = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT][run] for run in runs])))
    output = Category.parse_obj({
                "nodetype": "category",
                "input": "HLT",
                "content":[
                    CategoryItem.parse_obj({
                        "key": HLT, 
                        "value":Category.parse_obj({
                            "nodetype": "category",
                            "input": "run",
                            "content":[
                                CategoryItem.parse_obj({
                                    "key": run, 
                                    "value": getvalue(prescale_dict,year,HLT,run)
                                })
                                for run in list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT]])))
                            ],
                        })
                    })
                    for HLT in HLTs
                ],
    })
    return output


In [11]:
years=["2018"]

for year in years:
    corrs=[]
    corr = Correction.parse_obj(
        {
            "version": 2,
            "name": "HLT_prescale",
            "description": f"Testing prescale",
            "inputs": [
                {"name": "year","type": "string", "description": "year/scenario: example 2016preVFP, 2017 etc"},
                {"name": "HLT","type": "string", "description": "HLT name"},
                {"name": "run","type": "int", "description": "run "},
                {"name": "lumi","type": "int"},
            ],
            "output": {"name": "prescale", "type": "real", "description": "value of prescale"},
            "data": Category.parse_obj({
                "nodetype": "category",
                "input": "year",
                "content": [
                CategoryItem.parse_obj({"key":year,
                                        "value": getdictpre([year])})]
            })
        })

    corrs.append(corr)


    #Save JSON
    cset = CorrectionSet(schema_version=2, corrections=corrs,description=f"Trigger prescales- preliminary")
    with open("prescale.json", "w") as fout:
        fout.write(cset.json(exclude_unset=True, indent=4))

2018 HLT_Photon120 319503
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 315420
[28]
[98]
[28]
[1, 28, 99999]
[28]
[98]
[98]
[1, 98]
[1, 98]
2018 HLT_Photon120 317475
[33]
[128]
[33]
[1, 33, 99999]
[33]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 319524
[36]
[128]
[36]
[1, 36, 99999]
[36]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 317478
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 319526
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 319528
[6]
[128]
[6]
[1, 6, 99999]
[6]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 317484
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 317488
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 317527
[41]
[128]
[41]
[1, 41, 99999]
[41]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 319579
[41]
[128]
[41]
[1, 41, 99999]
[41]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 315488
[42]
[127]
[42]
[1, 42, 99999]
[42]


[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 322040
[32]
[128]
[32]
[1, 32, 99999]
[32]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 320002
[52]
[128]
[52]
[1, 52, 99999]
[52]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 320006
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 322057
[38]
[128]
[38]
[1, 38, 99999]
[38]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 320010
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 320011
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 320012
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 322068
[51]
[128]
[51]
[1, 51, 99999]
[51]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 320023
[17]
[128]
[17]
[1, 17, 99999]
[17]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 320024
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 320025
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Phot

[1, 128]
[1, 128]
2018 HLT_Photon120 320807
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 320809
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316715
[33]
[128]
[33]
[1, 33, 99999]
[33]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 316716
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316717
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316718
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316719
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316720
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316721
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316722
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 316723
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 320821
[41]
[128]
[41]
[1, 41, 99999]
[41]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT

[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 321432
[2]
[128]
[2]
[1, 2, 99999]
[2]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 321433
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 317338
[66]
[128]
[66]
[1, 66, 99999]
[66]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 317339
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 321436
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 317340
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 321434
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 323487
[42]
[128]
[42]
[1, 42, 99999]
[42]
[128]
[128]
[1, 128]
[1, 128]
2018 HLT_Photon120 323488
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 323492
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 323493
[1]
[128]
[1]
[1, 99999]
[1]
[128]
[128]
[128]
[128]
2018 HLT_Photon120 323495
[2]
[128]
[2]
[1, 2, 99999]
[2]
[128]
[1

[1]
[64]
[64]
[64]
[64]
2018 HLT_Photon175 321818
[1]
[64]
[1]
[1, 99999]
[1]
[64]
[64]
[64]
[64]
2018 HLT_Photon175 321820
[1]
[64]
[1]
[1, 99999]
[1]
[64]
[64]
[64]
[64]
2018 HLT_Photon175 321831
[25]
[64]
[25]
[1, 25, 99999]
[25]
[64]
[64]
[1, 64]
[1, 64]
2018 HLT_Photon175 321832
[1]
[64]
[1]
[1, 99999]
[1]
[64]
[64]
[64]
[64]
2018 HLT_Photon175 321833
[1]
[64]
[1]
[1, 99999]
[1]
[64]
[64]
[64]
[64]
2018 HLT_Photon175 315689
[25]
[63]
[25]
[1, 25, 99999]
[25]
[63]
[63]
[1, 63]
[1, 63]
2018 HLT_Photon175 315690
[10]
[63]
[10]
[1, 10, 99999]
[10]
[63]
[63]
[1, 63]
[1, 63]
2018 HLT_Photon175 321834
[4]
[64]
[4]
[1, 4, 99999]
[4]
[64]
[64]
[1, 64]
[1, 64]
2018 HLT_Photon175 315702
[39]
[63]
[39]
[1, 39, 99999]
[39]
[63]
[63]
[1, 63]
[1, 63]
2018 HLT_Photon175 315703
[1]
[63]
[1]
[1, 99999]
[1]
[63]
[63]
[63]
[63]
2018 HLT_Photon175 315704
[1]
[63]
[1]
[1, 99999]
[1]
[63]
[63]
[63]
[63]
2018 HLT_Photon175 315705
[1]
[63]
[1]
[1, 99999]
[1]
[63]
[63]
[63]
[63]
2018 HLT_Photon175 315713
[

...

2018 HLT_Photon120_R9Id90_HE10_IsoM 324970
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 318828
[54]
[8]
[54]
[1, 54, 99999]
[54]
[8]
[8]
[1, 8]
[1, 8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 324980
[39]
[8]
[39]
[1, 39, 99999]
[39]
[8]
[8]
[1, 8]
[1, 8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 320887
[49]
[8]
[49]
[1, 49, 99999]
[49]
[8]
[8]
[1, 8]
[1, 8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 320888
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 324997
[29]
[8]
[29]
[1, 29, 99999]
[29]
[8]
[8]
[1, 8]
[1, 8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 324998
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 324999
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 325000
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 325001
[1]
[8]
[1]
[1, 99999]
[1]
[8]
[8]
[8]
[8]
2018 HLT_Photon120_R9Id90_HE10_IsoM 320916
[2]
[8]
[2]
[1, 2, 99999]
[2]
[8]
[8]
[1, 8]


2018 HLT_Photon90_R9Id90_HE10_IsoM 319503
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 315420
[28]
[21]
[28]
[1, 28, 99999]
[28]
[21]
[21]
[1, 21]
[1, 21]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317475
[33]
[32]
[33]
[1, 33, 99999]
[33]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319524
[36]
[32]
[36]
[1, 36, 99999]
[36]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317478
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319526
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319528
[6]
[32]
[6]
[1, 6, 99999]
[6]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317484
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317488
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317527
[41]
[32]
[41]
[1, 41, 99999]
[41]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 3195

[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 322022
[42]
[32]
[42]
[1, 42, 99999]
[42]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 324077
[54]
[32]
[54]
[1, 54, 99999]
[54]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319991
[46]
[32]
[46]
[1, 46, 99999]
[46]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319992
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 319993
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 322040
[32]
[32]
[32]
[1, 32, 99999]
[32]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320002
[52]
[32]
[52]
[1, 52, 99999]
[52]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320006
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 322057
[38]
[32]
[38]
[1, 38, 99999]
[38]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320010
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32

[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320824
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320838
[93]
[32]
[93]
[1, 93, 99999]
[93]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320840
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320841
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320853
[41]
[32]
[41]
[1, 41, 99999]
[41]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 316758
[11]
[32]
[11]
[1, 11, 99999]
[11]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320854
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320855
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320856
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 320857
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_H

2018 HLT_Photon90_R9Id90_HE10_IsoM 321436
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 317340
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 321434
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 323487
[42]
[32]
[42]
[1, 42, 99999]
[42]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 323488
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 323492
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 323493
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 323495
[2]
[32]
[2]
[1, 2, 99999]
[2]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 321457
[43]
[32]
[43]
[1, 43, 99999]
[43]
[32]
[32]
[1, 32]
[1, 32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 321461
[1]
[32]
[1]
[1, 99999]
[1]
[32]
[32]
[32]
[32]
2018 HLT_Photon90_R9Id90_HE10_IsoM 315322
[23]
[15]
[23]
[1, 23, 

In [6]:
from correctionlib import _core

evaluator = _core.CorrectionSet.from_file('prescale.json')

In [9]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon120",317391,10.0)

RuntimeError: Input lumi has wrong type: got real-valued expected int

In [ ]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon120",315257,10.0)

In [ ]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon120",315257,10)

In [ ]:
2018 HLT_Photon165_R9Id90_HE10_IsoM 321887

In [ ]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon165_R9Id90_HE10_IsoM",321887,99.0)